In [1]:
import os
import sys
from typing import List


import torch
import transformers
from datasets import load_dataset

"""
Unused imports:
import torch.nn as nn
import bitsandbytes as bnb
"""

from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)
from peft import PeftModel
from transformers import LlamaForCausalLM, LlamaTokenizer

from utils.prompter import Prompter
# lora hyperparams
lora_r = 8
lora_alpha = 16
lora_dropout = 0.05
lora_target_modules = [
    "q_proj",
    "v_proj",
]

prompt_template_name = "alpaca"  # The prompt template to use, will default to alpaca.
device_map = "auto"

base_model = "/root/llama-7b-hf"  # the only required argument
data_path = "train_data_3_class_clean.jsonl"
output_dir = "/root/autodl-tmp/output"
lora_weights = "/root/autodl-tmp/output2s/checkpoint-100"
# resume_from_checkpoint = "/root/autodl-tmp/output/checkpoint-3700"
resume_from_checkpoint=None


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /root/miniconda3/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /root/miniconda3/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/root/miniconda3/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/root/miniconda3/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/root/miniconda3/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('Asia/Shanghai')}
  warn(msg)
/root/miniconda3/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/root/LLM-finetune/Untitled.ipynb')}
  warn(msg)


In [2]:
prompter = Prompter(prompt_template_name)
tokenizer = LlamaTokenizer.from_pretrained(base_model)

tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
 )
tokenizer.padding_side = "left"  # Allow batched inference

In [3]:
model = LlamaForCausalLM.from_pretrained(
        base_model,
        load_in_8bit=False,
        torch_dtype=torch.float16,
        device_map=device_map,
    )
model = PeftModel.from_pretrained(
    model,
    lora_weights,
    torch_dtype=torch.float16,
)


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [4]:
import json
from torch.utils.data import DataLoader
import time

# instructions = json.load(open("data/alpaca_data.json"))
instructions = []
with open(data_path, 'r') as f:
    for line in f:
        data = json.loads(line)
        data_point = {}
        data_point["instruction"] = 'What is the sentiment toward Bitcoin in the input sentence? [positive, negative, neutral]'
        data_point["input"] = data['text']
        data_point["output"] = data['label']
        full_prompt = prompter.generate_prompt(
            data_point["instruction"],
            data_point["input"],
            '',
        )
        instructions.append({'context':full_prompt, 'target':data['label']})

print(instructions[0])
start_time = time.time()
with torch.no_grad():
    right = 0
    all = 0
    batch_size = 8
    input_texts = []
    targets = []
    for idx, item in enumerate(instructions[300:]):
        # feature = format_example(item)
        # input_text = feature["context"]
        all = all + 1
        input_texts.append(item["context"])
        targets.append(item["target"])
    test_loader = DataLoader(input_texts, batch_size=batch_size)
    for batch_idx,batch in enumerate(test_loader):
        input_ids = tokenizer(batch, padding=True,return_tensors='pt').to('cuda')
        # out = model.generate(**input_ids, max_length=250, temperature=0)
        out = model.generate(
            **input_ids,
            temperature=0,
            return_dict_in_generate= True,
            output_scores=True,
            max_new_tokens = 1
        )
        seqs = out['sequences']
        scores = out['scores']
        # print(scores[0].shape)
        results = tokenizer.batch_decode(seqs)
        # print(results, '\n\n')
        # break
        for idx,res in enumerate(results):
            # print(res,'\n\n')
            pred = res[res.find('Response') + 10:]
            
            target = targets[batch_idx*batch_size + idx]
            # print(pred, target)
#             # print(pred, batch_idx*batch_size + idx,targets[batch_idx*batch_size + idx])\
            # if pred.find(target) >= 0:
            #     right = right + 1
            if target.find(pred) >= 0:
                right = right + 1
        print(right,all,right/all)
        # break
end_time = time.time()
elapsed_time = end_time - start_time
print("Finished in {:.2f} seconds.".format(elapsed_time))





{'context': "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is the sentiment toward Bitcoin in the input sentence? [positive, negative, neutral]\n\n### Input:\nBut... Bitcoin's halving IS priced in.\n\nFirst reminder of 2020\n\n### Response:\n", 'target': 'negative'}


/root/miniconda3/lib/python3.8/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


6 815 0.007361963190184049
13 815 0.015950920245398775
18 815 0.022085889570552148
23 815 0.02822085889570552
31 815 0.03803680981595092
38 815 0.046625766871165646
45 815 0.05521472392638037
52 815 0.0638036809815951
59 815 0.07239263803680981
65 815 0.07975460122699386
73 815 0.08957055214723926
79 815 0.09693251533742331
87 815 0.1067484662576687
94 815 0.11533742331288344
99 815 0.12147239263803682
106 815 0.13006134969325153
112 815 0.1374233128834356
119 815 0.1460122699386503
127 815 0.1558282208588957
134 815 0.16441717791411042
140 815 0.17177914110429449
148 815 0.18159509202453988
152 815 0.18650306748466258
158 815 0.19386503067484662
165 815 0.20245398773006135
170 815 0.2085889570552147
177 815 0.21717791411042944
185 815 0.22699386503067484
193 815 0.23680981595092024
201 815 0.24662576687116564
209 815 0.25644171779141106
215 815 0.26380368098159507
223 815 0.27361963190184047
231 815 0.28343558282208586
235 815 0.2883435582822086
240 815 0.294478527607362
248 815 0.304